<a href="https://colab.research.google.com/github/nimish1402/Amazon-Web-Scraper-/blob/main/Twitter_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#using twitters API

In [ ]:
import tweepy
import pandas as pd
import re
import os
import time

In [ ]:
API_KEY = "KJfBL1eL2ker6nhicKNhZRYch"
API_SECRET = "AMHdJbAgDgAY3I0yzb505nxiZAy9NSEr8bJnYfG7YGQw5wiOqs"
ACCESS_TOKEN = "1794412103232897024-F0Srvwq3HBVfQIVuGI45UJdCVY4vWN"
ACCESS_SECRET = "V1FlrHRM6A3Foo0pwnovT065M1nGcQSwg5Q0p0vycJeSZ"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAFzHygEAAAAA1hloN%2Bf4XNj2b6Dh0wUr5K2wjww%3Dllj5Ik3tp6USoSVHcGnU5hRdDJt8xmywFgpTig0BloS1z1ZqpT"

In [ ]:
client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [ ]:
# Load CSV file containing Twitter profile links
csv_file_path = "/content/Links.csv"
df = pd.read_csv(csv_file_path)


In [ ]:
df.head()

,Links
0,https://twitter.com/GTNUK1
1,https://twitter.com/whatsapp
2,https://twitter.com/aacb_CBPTrade
3,https://twitter.com/aacbdotcom
4,https://twitter.com/@AAWindowPRODUCT


In [ ]:
# Extract Twitter usernames from profile URLs
def extract_username(url):
    match = re.search(r"twitter\.com/([^/?]+)", str(url))
    return match.group(1) if match else None

df["Username"] = df["Links"].apply(extract_username)

In [ ]:
# List to store scraped data
data = []

In [ ]:
# Function to fetch user details with rate limit handling
def fetch_user_data(username):
    while True:
        try:
            user = client.get_user(username=username, user_fields=["description", "public_metrics", "location", "url"])
            return {
                "Username": username,
                "Bio": user.data.description if user.data.description else "",
                "Followers Count": user.data.public_metrics["followers_count"],
                "Following Count": user.data.public_metrics["following_count"],
                "Location": user.data.location if user.data.location else "",
                "Website": user.data.url if user.data.url else "",
            }
        except tweepy.TooManyRequests as e:
            reset_time = int(e.response.headers.get("x-rate-limit-reset", time.time() + 60))
            wait_time = reset_time - time.time()
            print(f"Rate limit exceeded. Waiting {wait_time:.0f} seconds...")
            time.sleep(wait_time + 1)
        except Exception as e:
            print(f"Error fetching data for {username}: {e}")
            return None

In [ ]:
for username in df["Username"].dropna().head(3):  # Process only the first 3 users
    user_data = fetch_user_data(username)
    if user_data:
        data.append(user_data)
        print(f"Fetched data for {username}")
        time.sleep(15)

Rate limit exceeded. Waiting 545 seconds...
Fetched data for GTNUK1
Fetched data for whatsapp
Fetched data for aacb_CBPTrade


In [ ]:
# Convert to DataFrame
output_df = pd.DataFrame(data)

# Define output path
output_dir = "/content/data "
output_csv_path = os.path.join(output_dir, "Twitter_Profile_Data.csv")

# Ensure directory exists
os.makedirs(output_dir, exist_ok=True)

# Save data to CSV (Append if file exists)
if os.path.exists(output_csv_path):
    output_df.to_csv(output_csv_path, mode="a", header=False, index=False)
else:
    output_df.to_csv(output_csv_path, index=False)

print(f"Data saved to {output_csv_path}")

Data saved to /content/data/Twitter_Profile_Data.csv


In [ ]:
df2 = pd.read_csv("/content/data/Twitter_Profile_Data.csv")

In [ ]:
df2.sample(5)

,Username,Bio,Followers Count,Following Count,Location,Website
1,whatsapp,message privately with WhatsApp,5595301,5,California,https://t.co/ku2ShnylHa
2,aacb_CBPTrade,Customs Broker🇺🇸,34,124,"Florida, USA",NaN
3,aacbdotcom,A & A Freight | Warehousing | Customs Brokerag...,651,3812,Worldwide,http://t.co/laZo1M99c7
6,Abrepro,From large format black & white prints to the ...,131,123,"Bentonville, AR",http://t.co/zyISURRqvP
4,aandb_kia,"A&B Kia is a Kia dealer in Benwood, WV. Stay c...",295,330,"Benwood, West Virginia",http://t.co/H1m1Bi9YHp
